In [1]:
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim

In [2]:
from Modelos import *
from Preprocessamento import *

2025-02-13 16:04:42.479949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-13 16:04:42.493127: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-13 16:04:42.497051: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 16:04:42.506679: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-13 16:04:43.334795: W tensorflow/compiler/tf2

In [3]:
treino_Kyoto, _ = preprocessamento_dataframe(caminho_csv='CSV/Kyoto/Kyoto_Segmentado_Treino.csv', autoencoder=True)
validacao_Kyoto, _ = preprocessamento_dataframe(caminho_csv='CSV/Kyoto/Kyoto_Segmentado_Validacao.csv', autoencoder=True)
teste_Kyoto, _ = preprocessamento_dataframe(caminho_csv='CSV/Kyoto/Kyoto_Segmentado_Teste.csv', autoencoder=True, data_algumentantation=False)

Found 44 validated image filenames.
Imagens totais: 44
Found 10 validated image filenames.
Imagens totais: 10
Found 8 validated image filenames.
Imagens totais: 8


In [4]:
from tensorflow.keras import backend as K
import tensorflow.image as tf_img

def calcular_mse(image1, image2):
    # Measures the average squared difference between corresponding pixels in two images. This is a basic pixel-wise comparison metric.
    # 1 ruim, 0 bom
    return K.mean(K.square(image1 - image2)).numpy()

def calcular_ssim(image1, image2):
    # Measures the perceptual difference between two images by comparing luminance, contrast, and structure.
    # -1 ruim, 1 bom
    return tf_img.ssim(image1, image2, max_val=1.0).numpy()

def calcular_psnr(image1, image2):
    # Measures the ratio between the maximum possible pixel value and the power of the noise (difference) between two images. It’s a logarithmic scale.
    # 30-50 dB: Indicates high quality. A PSNR value in this range means the images are quite similar.
    # 20-30 dB: Moderate quality. There is noticeable difference, but the images are still somewhat similar.
    # < 20 dB: Low quality, with significant differences between the two images.
    return tf.image.psnr(image1, image2, max_val=1.0)

def calcular_ncc(image1, image2):
    # Measures the similarity between two images by comparing how well they align after normalization. It's often used for template matching.
    # NCC = +1: Maximum similarity (no difference).
    # NCC = 0: No correlation (the images are unrelated).
    # NCC = -1: Maximum negative correlation (complete inverse/opposites).
    image1_mean = np.mean(image1)
    image2_mean = np.mean(image2)
    numerator = np.sum((image1 - image1_mean) * (image2 - image2_mean))
    denominator = np.sqrt(np.sum((image1 - image1_mean)**2) * np.sum((image2 - image2_mean)**2))
    return numerator / denominator

def calcular_perceptual(image1, image2):
    # Used to match the style of one image to another while preserving the content of the original.
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    image1 = preprocess_input(tf.image.resize(image1, (224, 224)))
    image2 = preprocess_input(tf.image.resize(image2, (224, 224)))
    features1 = model(tf.expand_dims(image1, axis=0))
    features2 = model(tf.expand_dims(image2, axis=0))
    return tf.reduce_mean(tf.square(features1 - features2))


In [13]:
gerador = Gerador()
gerador.Dataset(treino_Kyoto, validacao_Kyoto, teste_Kyoto)

x, y = next(teste_Kyoto)
for i in range(10):
    modelo = gerador.carrega_modelo(f'Modelos/Modelo_Kyoto-{i}/Modelo-Base/Estrutura/Modelo_Kyoto-{i}.keras', f'Modelos/Modelo_Kyoto-{i}/Modelo-Base/Pesos/Modelo_Kyoto-{i}_Base-Kyoto.weights.h5')
    reconstructed_images = gerador.autoencoder.predict(x)
    
    mse = calcular_mse(x, reconstructed_images)
    ssim = calcular_ssim(x, reconstructed_images)
    psnr = calcular_psnr(x, reconstructed_images)

    if mse > 0.1 or ssim[0] < 0.5 or psnr[0] < 20:
        print("Ruim")
    else:
        print("Bom")


Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 1849)           │     7,576,745 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 64, 64, 3)      │     7,581,299 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,158,044 (57.82 MB)

 Trainable params: 15,158,044 (57.82 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 148)            │        41,532 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 64, 64, 3)      │        50,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 92,263 (360.40 KB)

 Trainable params: 92,263 (360.40 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
Ruim
